In [1]:
import pandas as pd
import glob
import math
import numpy as np

In this notebook we show to locally evaluate a submission generated from a subset of the csv's from your training data.

In [2]:
data_path = 'data/mini/' # TODO: point this to your data folder
submission_path = data_path + 'submissions/'
training_path = data_path + 'training_set/'
input_logs = sorted(glob.glob(training_path + "log_mini*.csv")) # TODO: point this to a subset of csv's in your training set

In [3]:
input_logs

['data/mini/training_set/log_mini.csv']

In [4]:
df = pd.read_csv(input_logs[0])

In [5]:
len(df)

167880

In [6]:
df.sample(10)

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
103738,0_090cdf38-8f16-4234-bd60-03642168ba44,6,20,t_0ef6d2c8-faf9-4ebe-ac75-2bade8696a3d,True,True,True,False,0,1,...,0,0,0,False,12,2018-07-15,True,catalog,backbtn,backbtn
121750,0_0ab2b461-4ab3-404f-a1a4-79a13811cd77,3,20,t_495ffcba-e460-47c3-810d-ac4e5edab830,False,False,False,True,0,1,...,0,0,1,False,23,2018-07-14,True,user_collection,clickrow,trackdone
142755,0_0c83dfb8-7883-4c53-8f81-7347a1027feb,1,19,t_69c9092f-cdf3-4f27-ac1e-42c929ac5e32,False,False,False,True,0,0,...,0,0,0,False,10,2018-07-15,True,user_collection,fwdbtn,trackdone
111144,0_09ae0719-0940-4f44-a737-52df5714ae1f,1,20,t_5954ccc3-11a6-4f32-a886-bafe96643ab9,False,False,True,False,0,0,...,0,0,0,True,13,2018-07-15,True,user_collection,trackdone,endplay
7255,0_009b098d-99f8-4019-8574-881dfbc1de4e,8,13,t_a66ea088-b357-449a-8a1e-64dd0b8d6cb5,True,True,True,False,0,1,...,0,0,0,False,21,2018-07-14,False,user_collection,backbtn,fwdbtn
35379,0_03154981-7c89-48e5-bff6-6115e654bc42,12,20,t_8a30cd7e-ea72-4677-a959-a81da9a55577,True,True,True,False,0,0,...,1,0,0,True,14,2018-07-15,True,user_collection,fwdbtn,fwdbtn
110107,0_09999fcf-822a-4a80-a320-88d97e5a2599,18,20,t_abe577df-bc3e-48ff-a13c-f2fd7a295ba6,False,False,False,True,0,1,...,0,0,0,False,22,2018-07-14,False,editorial_playlist,trackdone,trackdone
18500,0_0196f19a-3229-4174-b1c4-6073d9a7c67d,6,14,t_7cebd022-6b41-434e-8021-9cce9ba43115,True,True,True,False,0,1,...,0,0,0,False,15,2018-07-15,False,user_collection,fwdbtn,fwdbtn
154580,0_0d8b1c9e-2d20-437c-ae6e-af89dab12434,4,15,t_a7d9394f-8e60-4746-b4d9-06862f8115d5,True,True,True,False,0,1,...,0,0,0,False,18,2018-07-14,False,user_collection,fwdbtn,fwdbtn
115793,0_0a24e6dc-687c-4c3a-9b97-40e78ea4add3,1,20,t_18573083-f940-420c-b9fd-2f996faa56b6,True,True,True,False,0,0,...,0,0,0,False,20,2018-07-14,False,editorial_playlist,fwdbtn,fwdbtn


In [7]:
def get_ground_truth(input_logs):
    ground_truths = []     
    for i,f in enumerate(input_logs):
        df = pd.read_csv(f)
        # Below we keep only the relevant columns of the second half of the session for saving the ground truth
        df = df[['session_id','skip_2','session_position','session_length']].loc[df['session_position']*2 > df['session_length']]
        df = df.reset_index()
        current_index = 0
        # Here we process each session, saving a list containing the 
        while current_index < len(df):
            partial_length = df['session_length'].iloc[current_index]-df['session_position'].iloc[current_index]+1
            session_skips = list(df.loc[current_index:current_index+partial_length-1, 'skip_2'])
            ground_truths.append(session_skips)
            current_index += partial_length 
    return ground_truths

In [8]:
ground_truth = get_ground_truth(input_logs)

In [9]:
ground_truth

[[True, True, True, False, True, True, True, True, True, True],
 [False, False, True, False, True, True, False, False, False, False],
 [False, True, False, False, True, True, True, True, True, True],
 [True, True, True, True, True, True, False, True, True, True],
 [False, False, False, False, False, False],
 [False, False, False, True, False, False, False, False],
 [True, True, False, False, False, False, False, False],
 [False, False, False, False, False, False, False, False, False, False],
 [False, False, False, False, False, False, False, False, True, False],
 [False, True, False, False, False, False, False, False, False, False],
 [True, True, True, True, True, True, True, True, True, True],
 [True, False, False, False, False, False, False, False, False, False],
 [True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, False, False],
 [True, True, True, True, False, False, False, False, True, False],
 [True, False, False, False,

In [10]:
len(ground_truth)

10000

In [11]:
def generate_random_submission(input_logs):
    output = []
    for i,f in enumerate(input_logs):
        df = pd.read_csv(f)
        print('file {} read'.format(i))
        # For this random submission we just need to know the length of each session
        df = df.drop_duplicates(subset='session_id', keep='first', inplace=False)
        # For each session we generate a random prediction of the required length
        for j in df.index:
            sess_len = df.loc[j,'session_length']
            partial_len = math.ceil(sess_len/2)
            current_preds = np.random.choice([0, 1], size=(partial_len,))
            output.append(current_preds)
    return output

In [12]:
submission = generate_random_submission(input_logs)

file 0 read


In [13]:
def evaluate(submission,groundtruth):
    ap_sum = 0.0
    first_pred_acc_sum = 0.0
    counter = 0
    for sub, tru in zip(submission, groundtruth):
        if len(sub) != len(tru):
            raise Exception('Line {} should contain {} predictions, but instead contains '
                            '{}'.format(counter+1,len(tru),len(sub)))
        ap_sum += ave_pre(sub,tru,counter)
        first_pred_acc_sum += sub[0] == tru[0]
        counter+=1
    ap = ap_sum/counter
    first_pred_acc = first_pred_acc_sum/counter
    return ap,first_pred_acc

In [14]:
def ave_pre(submission,groundtruth,counter):
    s = 0.0
    t = 0.0
    c = 1.0
    for x, y in zip(submission, groundtruth):
        if x != 0 and x != 1:
            raise Exception('Invalid prediction in line {}, should be 0 or 1'.format(counter))
        if x==y:
            s += 1.0
            t += s / c
        c += 1
    return t/len(groundtruth)

In [15]:
ap,first_pred_acc = evaluate(submission,ground_truth)

In [16]:
print('average precision: {}'.format(ap))
print('first prediction accuracy: {}'.format(first_pred_acc))

average precision: 0.3325479414525066
first prediction accuracy: 0.4946
